In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf

# Import project modules
from conllu_reader import ConlluReader
from algorithm import ArcEager, Transition, Sample
from conllu_token import Token
from model import ParserMLP

# Check for GPU availability (Optional)
print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-11-19 17:31:42.863863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-19 17:31:42.967021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-19 17:31:43.002124: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-19 17:31:43.172370: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow Version: 2.17.0
Num GPUs Available:  1


I0000 00:00:1763573507.747241     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763573508.170321     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763573508.170355     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
dataset_path = "dataset.pkl"

if not os.path.exists(dataset_path):
    print(f"Error: '{dataset_path}' not found. Please run main.py first to generate it.")
else:
    print(f"Loading dataset from {dataset_path}...")
    with open(dataset_path, "rb") as f:
        data = pickle.load(f)
        
    training_samples = data["training_samples"]
    dev_samples = data["dev_samples"]
    deprels = data["deprels"]
    actions = data["actions"]
    
    print("Dataset loaded successfully!")
    print(f"Training Samples: {len(training_samples)}")
    print(f"Development Samples: {len(dev_samples)}")
    print(f"Unique Dependency Labels: {len(deprels)}")

Loading dataset from dataset.pkl...
Dataset loaded successfully!
Training Samples: 81182
Development Samples: 4978
Unique Dependency Labels: 43


In [11]:
# Inspect the first training sample
sample = training_samples[80000]

print(f"--- Sample 1 ---")
# Note: Accessing internal state for visualization
print(f"State Stack: {[t.form for t in sample.state.S]}")
print(f"State Buffer (first 3): {[t.form for t in sample.state.B[:3]]}...")
print(f"Gold Transition: {sample.transition}")

--- Sample 1 ---
State Stack: ['ROOT', 'enlisted', '.']
State Buffer (first 3): []...
Gold Transition: LEFT-ARC-det


In [ ]:
# Extract features from the loaded sample
feats = sample.state_to_feats(nstack_feats=2, nbuffer_feats=2)

print("Feature List:", feats)

# Expected output format: 
# [Stack_Word_1, Stack_Word_0, Buffer_Word_0, Buffer_Word_1, 
#  Stack_UPOS_1, Stack_UPOS_0, Buffer_UPOS_0, Buffer_UPOS_1]

Feature List: ['.', 'enlisted', '<PAD>', '<PAD>', 'PUNCT', 'VERB', '<PAD>', '<PAD>']


In [13]:
# Initialize Model
# You can adjust dimensions and epochs as needed
model = ParserMLP(word_emb_dim=100, hidden_dim=64, epochs=3, batch_size=32)

print("Starting training...")
# The train function handles vocabulary building and vectorization internally
model.train(training_samples, dev_samples)

Starting training...
Building vocabulary...
Vocab built: 1004 words, 16 UPOS tags.
Outputs: 4 actions, 43 dependency labels.
Vectorizing data...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 words_input (InputLayer)    [(None, 4)]                  0         []                            
                                                                                                  
 upos_input (InputLayer)     [(None, 4)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 4, 100)               100400    ['words_input[0][0]']         
                                                                                                  
 embedding_1 (Embedding)     (None, 4, 50)      

I0000 00:00:1763573633.476557     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763573633.476613     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763573633.476623     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763573633.766474     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763573633.766531     140 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-19

Epoch 1/3


'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring f

   3/2537 [..............................] - ETA: 3:08 - loss: 3.2448 - action_output_loss: 1.3756 - deprel_output_loss: 3.7384 - action_output_accuracy: 0.3229 - deprel_output_accuracy: 0.1771  

...

2199/2537 [=========================>....] - ETA: 8s - loss: 2.5002 - action_output_loss: 1.3725 - deprel_output_loss: 2.2552 - action_output_accuracy: 0.2986 - deprel_output_accuracy: 0.4858 

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


2537/2537 [==============================] - 28s 10ms/step - loss: 2.4963 - action_output_loss: 1.3720 - deprel_output_loss: 2.2487 - action_output_accuracy: 0.2984 - deprel_output_accuracy: 0.4868 - val_loss: 2.4778 - val_action_output_loss: 1.3579 - val_deprel_output_loss: 2.2398 - val_action_output_accuracy: 0.3055 - val_deprel_output_accuracy: 0.4767
Epoch 2/3
2422/2537 [===========================>..] - ETA: 2s - loss: 2.4629 - action_output_loss: 1.3615 - deprel_output_loss: 2.2027 - action_output_accuracy: 0.3009 - deprel_output_accuracy: 0.4874 

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


2537/2537 [==============================] - 27s 11ms/step - loss: 2.4632 - action_output_loss: 1.3615 - deprel_output_loss: 2.2034 - action_output_accuracy: 0.3011 - deprel_output_accuracy: 0.4872 - val_loss: 2.4759 - val_action_output_loss: 1.3580 - val_deprel_output_loss: 2.2359 - val_action_output_accuracy: 0.3043 - val_deprel_output_accuracy: 0.4767
Epoch 3/3
2537/2537 [==============================] - 42s 17ms/step - loss: 2.4514 - action_output_loss: 1.3592 - deprel_output_loss: 2.1845 - action_output_accuracy: 0.3002 - deprel_output_accuracy: 0.4872 - val_loss: 2.4780 - val_action_output_loss: 1.3622 - val_deprel_output_loss: 2.2315 - val_action_output_accuracy: 0.2995 - val_deprel_output_accuracy: 0.4767


In [14]:
# 1. Load a sentence for testing (e.g., from the dev set)
reader = ConlluReader()
dev_trees = reader.read_conllu_file("en_partut-ud-dev_clean.conllu", inference=False)
dev_trees = reader.remove_non_projective_trees(dev_trees)

# Pick a sentence
original_tree = dev_trees[0]
input_tokens = []

# Create a clean version of the tokens (simulating inference mode)
for t in original_tree:
    new_token = Token(t.id, t.form, t.lemma, t.upos, t.cpos, t.feats, "_", "_") 
    input_tokens.append(new_token)

# 2. Run the model
print("Running inference on sentence:")
print(" ".join([t.form for t in input_tokens if t.id != 0]))

# Pass as a list of sentences (even if just one)
parsed_sents = model.run([input_tokens])
parsed_tree = parsed_sents[0]

# 3. Inspect Results
print("\nParsed Result:")
print(f"{'ID':<3} {'FORM':<15} {'HEAD':<5} {'DEPREL'}")
print("-" * 40)
for t in parsed_tree:
    if t.id == 0: continue
    print(f"{t.id:<3} {t.form:<15} {t.head:<5} {t.dep}")

# 4. Compare with Gold Standard
correct_heads = 0
total_tokens = 0
for pred, gold in zip(parsed_tree, original_tree):
    if pred.id == 0: continue
    total_tokens += 1
    if str(pred.head) == str(gold.head):
        correct_heads += 1

uas = correct_heads / total_tokens if total_tokens > 0 else 0
print(f"\nUnlabeled Attachment Score (UAS): {uas:.2f}")

Running inference on sentence:
Creative Commons Corporation is not a law firm and does not provide legal services .

Parsed Result:
ID  FORM            HEAD  DEPREL
----------------------------------------
1   Creative        _     _
2   Commons         _     _
3   Corporation     _     _
4   is              _     _
5   not             _     _
6   a               _     _
7   law             _     _
8   firm            _     _
9   and             _     _
10  does            11    acl
11  not             12    acl
12  provide         _     _
13  legal           14    acl
14  services        15    acl
15  .               _     _

Unlabeled Attachment Score (UAS): 0.13
